In [ ]:
# 트위터 간단한 텍스트마이닝 분류기법
# 작성자 : Fairies / AdonisHan
# Naive Bayes 기법


In [30]:
#import math, sys
from konlpy.tag import Twitter
import konlpy
import bayes

In [31]:
class BayesianFilter:
    def __init__(self):
        self.words = set() #출현한 단어 기록
        self.word_dict = {} #카테고리마다 출현 횟수 기록
        self.category_dict = {} #카테고리 출현 횟수 기록

    # 형태소 분석
    def split(self, text):
        results = []
        twitter = Twitter()

        #단어의 기본형 사용
        malist = twitter.pos(text, norm = True, stem = True)
        for word in malist:
            #어미, 조사, 구두점 등은 제외하기
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                results.append(word[0])
        return results

    #단어와 카테고리의 출현 횟수 세기
    def inc_word(self, word, category):
        #단어를 카테고리에 추가
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)

    def inc_category(self, category):
        #카테고리 계산
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1

    #텍스트 학습하기
    def fit(self, text, category):
        # 텍스트 학습
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)

    #단어 리스트에 점수매기기 - log
    def score(self, words, category):
        score = math.log(self.category_prob(category)) #log화, downflow 예방
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score

    #예측
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        words = self.split(text)
        socre_list = []
        for category in self.category_dict.keys():
            score = self.score(words, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, socre_list

    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0

    # 카테고리 계산하기
    def cateogry_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories

    # 카테고리 내부의 단어 출현 비율 계산
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 #smoothing
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d

In [ ]:
#NB test
bf = BayesianFilter()

bf.fit("파격 세일 - 오늘까지만 30% 할인", "광고")
bf.fit("쿠폰 선물", "광고")
bf.fit("멋진 블루투스 스피커", "광고")
bf.fit("메세지가 한 건", "중요")
bf.fit("오늘 일정 9시 회의", "중요")

# 예측
pre.scorelist = bf.predict("세일 할인, 선물")
print("결과", pre)
print(scorelist)